# Итоговый проект. Улицы. Объекты культурного наследия.

In [ ]:
# установка всех необходимых модулей

!pip install geopandas mapclassify osmnx keplergl

In [36]:
# импорты всех модулей
import json
import geopandas as gpd
import osmnx as ox
from google.colab import output
from keplergl import KeplerGl

In [34]:
output.enable_custom_widget_manager() # формально это не импорт, нужно для Кеплера

In [46]:
TILES = "CartoDB positron"  # Название подложки для карт

# район Санкт-Петербург из OSM https://wiki.openstreetmap.org/wiki/RU:%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3/%D0%A0%D0%B0%D0%B9%D0%BE%D0%BD%D1%8B
TERRITORY_NAME = 'Адмиралтейский район'   # название территории для которой будут строиться слои

# TODO указать ссылку на файл из вашего github репозитория, которая начинается с https://raw.githubusercontent.com/
KGIOP_FILE_URL = "https://raw.githubusercontent.com/aeksei/PythonUrbanITMO2023/main/geojson_layers/kgiop_objects.geojson"  # ссылка на слой с объектами культурного наследия
STREETS_FILE_URL = "https://raw.githubusercontent.com/aeksei/PythonUrbanITMO2023/main/geojson_layers/streets.geojson"  # ссылка на слой с улицами

## Территория

### Загрузка территории из OSM (Extract)

In [4]:
# загружен geodataframe с геометрией для территории TERRITORY_NAME

district = ox.geocode_to_gdf(TERRITORY_NAME)
district

,geometry,bbox_north,bbox_south,bbox_east,bbox_west,place_id,osm_type,osm_id,lat,lon,class,type,place_rank,importance,addresstype,name,display_name
0,"POLYGON ((30.25023 59.90129, 30.25112 59.90103...",59.941057,59.896031,30.342678,30.250229,181631755,relation,1114193,59.918443,30.302795,boundary,administrative,10,0.41336,administrative,Admiralteysky District,"Admiralteysky District, Сенной округ, Saint Pe..."


In [5]:
district.explore()

## Улицы

### Загрузка файла с улицами (Extract)

In [16]:
# отфильтровать улицы по маске геометрии территории полученной ранее

streets = gpd.read_file(STREETS_FILE_URL, mask=district)
streets

,name,geometry
0,набережная Обводного канала,"LINESTRING (3368853.534 8379613.768, 3368894.2..."
1,переулок Гривцова,"LINESTRING (3374917.530 8383417.375, 3374910.2..."
2,Люблинский переулок,"LINESTRING (3372221.049 8381344.575, 3372211.4..."
3,Бумажная улица,"LINESTRING (3369358.324 8379129.066, 3369358.6..."
4,13-я Красноармейская улица,"LINESTRING (3372683.994 8380856.184, 3372703.9..."
...,...,...
1184,Малая Митрофаньевская улица,"LINESTRING (3374305.629 8376701.087, 3374290.3..."
1185,Малая Митрофаньевская улица,"LINESTRING (3374263.773 8376690.877, 3374279.6..."
1186,Парфёновская улица,"LINESTRING (3374321.804 8378141.245, 3374321.6..."
1187,набережная реки Фонтанки,"LINESTRING (3375466.001 8382603.187, 3375113.7..."


In [18]:
streets[streets['name']=='Малая Митрофаньевская улица']

,name,geometry
64,Малая Митрофаньевская улица,"LINESTRING (3373001.532 8376648.553, 3373025.5..."
99,Малая Митрофаньевская улица,"LINESTRING (3374154.468 8376679.292, 3374166.3..."
110,Малая Митрофаньевская улица,"LINESTRING (3375115.200 8376714.315, 3375111.6..."
611,Малая Митрофаньевская улица,"LINESTRING (3374511.114 8376696.826, 3374622.9..."
1038,Малая Митрофаньевская улица,"LINESTRING (3373270.759 8376661.093, 3373384.5..."
1039,Малая Митрофаньевская улица,"LINESTRING (3373270.759 8376661.093, 3373170.4..."
1177,Малая Митрофаньевская улица,"LINESTRING (3374316.149 8376691.454, 3374494.1..."
1182,Малая Митрофаньевская улица,"LINESTRING (3374325.444 8376701.220, 3374305.6..."
1183,Малая Митрофаньевская улица,"LINESTRING (3374511.114 8376696.826, 3374493.9..."
1184,Малая Митрофаньевская улица,"LINESTRING (3374305.629 8376701.087, 3374290.3..."


### Обработка данных с улицами (Transform)

In [31]:
# сгруппировать и объединить геометрии с одинаковыми названиями
streets = streets.dissolve(by="name")
streets

,geometry
name,
1-й мост Круштейна,"MULTILINESTRING ((3371922.223 8384393.995, 337..."
1-я Красноармейская улица,"MULTILINESTRING ((3373826.198 8381061.512, 337..."
10-я Красноармейская улица,"MULTILINESTRING ((3372133.931 8380334.118, 337..."
11-я Красноармейская улица,"MULTILINESTRING ((3373790.832 8380234.239, 337..."
12-я Красноармейская улица,"MULTILINESTRING ((3372708.718 8380047.963, 337..."
...,...
улица Степана Разина,"MULTILINESTRING ((3369019.100 8379612.857, 336..."
улица Труда,"LINESTRING (3372477.752 8383885.875, 3372458.3..."
улица Циолковского,"MULTILINESTRING ((3371725.332 8379587.412, 337..."


In [37]:
map = KeplerGl(height=500)
map.add_data(data=streets.copy(), name="Улицы")  # почему-то метод add_data изменяет геодатафрейм :( будем делать копию

map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'Улицы': {'index': ['1-й мост Круштейна', '1-я Красноармейская улица', '10-я Красноармейская ул…

### Сохранение слоя с улицами (Load)

In [39]:
# переименовать столбцы в русские названия, кроме столбца geometry (там всего один столбец и это geometry)

streets.index.names = ['Название_улицы']

In [43]:
streets.head()

,geometry
Название_улицы,
1-й мост Круштейна,"MULTILINESTRING ((3371922.223 8384393.995, 337..."
1-я Красноармейская улица,"MULTILINESTRING ((3373826.198 8381061.512, 337..."
10-я Красноармейская улица,"MULTILINESTRING ((3372133.931 8380334.118, 337..."
11-я Красноармейская улица,"MULTILINESTRING ((3373790.832 8380234.239, 337..."
12-я Красноармейская улица,"MULTILINESTRING ((3372708.718 8380047.963, 337..."


In [44]:
# сохраним слой в географической проекции в формате GeoJSON
streets.to_crs(4326).to_file('streets.geojson', driver='GeoJSON')

## Объекты культурного наследия

### Загрузка объектов культурного наследия (Extract)

In [71]:
# отфильтровать ОКН по маске геометрии территории полученной ранее

heritage = gpd.read_file(KGIOP_FILE_URL, mask=district)
heritage.head()

,id,ensemble_name,object_name,occurrence_time,object_location,historical_category,normative_act,object_type,geometry
0,1,—,Здание Консисторского управления Могилевской Р...,1870-1873; 1878-1879; 1896-1897; 1900-1902,"1-я Красноармейская ул., 11, лит. А, Б",объект культурного наследия регионального знач...,Распоряжение КГИОП № 10-22 от 21.07.2009,Памятник,POINT (3374338.001 8381192.049)
1,2,—,Здание манежа (экзерциргауса) лейб-гвардии Изм...,1795-1797,"1-я Красноармейская ул., 13; Измайловский пр.,...",объект культурного наследия регионального знач...,Закон Санкт-Петербурга № 141-47 от 02.07.1997,Памятник,POINT (3373995.026 8381101.886)
2,3,—,"Дом, где в начале 1895 г. Ленин В.И. встречалс...",—,"1-я Красноармейская ул., 22",объект культурного наследия регионального знач...,Закон Санкт-Петербурга № 141-47 от 02.07.1997,Памятник,POINT (3374182.710 8381041.925)
3,4,—,Трансформаторная подстанция,1906-1907,"11-я Красноармейская ул., 28",объект культурного наследия регионального знач...,Решение исполкома Ленгорсовета № 963 от 05.12....,Памятник,POINT (3372847.990 8380161.229)
4,5,—,Манеж графа Г.И. Рибопьера (Здание Санкт-Петер...,1887; 1891,"2-я Красноармейская ул., 12",объект культурного наследия регионального знач...,Распоряжение КГИОП № 10-26 от 15.09.2009,Памятник,POINT (3374362.046 8380882.255)


In [72]:
heritage.shape

(1022, 9)

In [ ]:
# можно заметить, что координаты точек даны в метрической системе

In [73]:
# способ 1 - сохраним в нужном формате и прочитаем заново
heritage.to_crs(4326).to_file('heritage.geojson', driver='GeoJSON')

In [74]:
heritage4326 = gpd.read_file('heritage.geojson', mask=district)
heritage4326.head()

,id,ensemble_name,object_name,occurrence_time,object_location,historical_category,normative_act,object_type,geometry
0,1,—,Здание Консисторского управления Могилевской Р...,1870-1873; 1878-1879; 1896-1897; 1900-1902,"1-я Красноармейская ул., 11, лит. А, Б",объект культурного наследия регионального знач...,Распоряжение КГИОП № 10-22 от 21.07.2009,Памятник,POINT (30.31219 59.91660)
1,2,—,Здание манежа (экзерциргауса) лейб-гвардии Изм...,1795-1797,"1-я Красноармейская ул., 13; Измайловский пр.,...",объект культурного наследия регионального знач...,Закон Санкт-Петербурга № 141-47 от 02.07.1997,Памятник,POINT (30.30911 59.91619)
2,3,—,"Дом, где в начале 1895 г. Ленин В.И. встречалс...",—,"1-я Красноармейская ул., 22",объект культурного наследия регионального знач...,Закон Санкт-Петербурга № 141-47 от 02.07.1997,Памятник,POINT (30.31080 59.91592)
3,4,—,Трансформаторная подстанция,1906-1907,"11-я Красноармейская ул., 28",объект культурного наследия регионального знач...,Решение исполкома Ленгорсовета № 963 от 05.12....,Памятник,POINT (30.29881 59.91195)
4,5,—,Манеж графа Г.И. Рибопьера (Здание Санкт-Петер...,1887; 1891,"2-я Красноармейская ул., 12",объект культурного наследия регионального знач...,Распоряжение КГИОП № 10-26 от 15.09.2009,Памятник,POINT (30.31241 59.91520)


### Обработка объектов культурного наследия (Transform)

In [76]:
# способ 2
heritage.to_crs(4326).head()

,id,ensemble_name,object_name,occurrence_time,object_location,historical_category,normative_act,object_type,geometry
0,1,—,Здание Консисторского управления Могилевской Р...,1870-1873; 1878-1879; 1896-1897; 1900-1902,"1-я Красноармейская ул., 11, лит. А, Б",объект культурного наследия регионального знач...,Распоряжение КГИОП № 10-22 от 21.07.2009,Памятник,POINT (30.31219 59.91659)
1,2,—,Здание манежа (экзерциргауса) лейб-гвардии Изм...,1795-1797,"1-я Красноармейская ул., 13; Измайловский пр.,...",объект культурного наследия регионального знач...,Закон Санкт-Петербурга № 141-47 от 02.07.1997,Памятник,POINT (30.30911 59.91619)
2,3,—,"Дом, где в начале 1895 г. Ленин В.И. встречалс...",—,"1-я Красноармейская ул., 22",объект культурного наследия регионального знач...,Закон Санкт-Петербурга № 141-47 от 02.07.1997,Памятник,POINT (30.31080 59.91592)
3,4,—,Трансформаторная подстанция,1906-1907,"11-я Красноармейская ул., 28",объект культурного наследия регионального знач...,Решение исполкома Ленгорсовета № 963 от 05.12....,Памятник,POINT (30.29881 59.91195)
4,5,—,Манеж графа Г.И. Рибопьера (Здание Санкт-Петер...,1887; 1891,"2-я Красноармейская ул., 12",объект культурного наследия регионального знач...,Распоряжение КГИОП № 10-26 от 15.09.2009,Памятник,POINT (30.31241 59.91520)


In [77]:
#  добавить два столбца lon и lat, в которых будут долгота и широта
heritage['lon'] = heritage['geometry'].x
heritage['lat'] = heritage['geometry'].y

In [78]:
heritage.head()

,id,ensemble_name,object_name,occurrence_time,object_location,historical_category,normative_act,object_type,geometry,lon,lat
0,1,—,Здание Консисторского управления Могилевской Р...,1870-1873; 1878-1879; 1896-1897; 1900-1902,"1-я Красноармейская ул., 11, лит. А, Б",объект культурного наследия регионального знач...,Распоряжение КГИОП № 10-22 от 21.07.2009,Памятник,POINT (3374338.001 8381192.049),3.374338e+06,8.381192e+06
1,2,—,Здание манежа (экзерциргауса) лейб-гвардии Изм...,1795-1797,"1-я Красноармейская ул., 13; Измайловский пр.,...",объект культурного наследия регионального знач...,Закон Санкт-Петербурга № 141-47 от 02.07.1997,Памятник,POINT (3373995.026 8381101.886),3.373995e+06,8.381102e+06
2,3,—,"Дом, где в начале 1895 г. Ленин В.И. встречалс...",—,"1-я Красноармейская ул., 22",объект культурного наследия регионального знач...,Закон Санкт-Петербурга № 141-47 от 02.07.1997,Памятник,POINT (3374182.710 8381041.925),3.374183e+06,8.381042e+06
3,4,—,Трансформаторная подстанция,1906-1907,"11-я Красноармейская ул., 28",объект культурного наследия регионального знач...,Решение исполкома Ленгорсовета № 963 от 05.12....,Памятник,POINT (3372847.990 8380161.229),3.372848e+06,8.380161e+06
4,5,—,Манеж графа Г.И. Рибопьера (Здание Санкт-Петер...,1887; 1891,"2-я Красноармейская ул., 12",объект культурного наследия регионального знач...,Распоряжение КГИОП № 10-26 от 15.09.2009,Памятник,POINT (3374362.046 8380882.255),3.374362e+06,8.380882e+06


### Сохранение слоя с объектами культурного наследия (Load)

In [69]:
# переименовать столбцы в русские названия, кроме столбца geometry
heritage = heritage.rename(columns={'ensemble_name': 'название_ансамбля',
                          'object_name': 'название_объекта',
                          'occurrence_time': 'год_создания',
                          'object_location': 'местоположение_объекта',
                          'historical_category': 'историческая_категория',
                          'normative_act': 'историческая_категория',
                          'object_type': 'историческая_категория',
                          'lon': 'широта',
                          'lat': 'долгота'
                          })

In [70]:
heritage

,id,название_ансамбля,название_объекта,год_создания,местоположение_объекта,историческая_категория,историческая_категория,историческая_категория,geometry,широта,долгота
0,1,—,Здание Консисторского управления Могилевской Р...,1870-1873; 1878-1879; 1896-1897; 1900-1902,"1-я Красноармейская ул., 11, лит. А, Б",объект культурного наследия регионального знач...,Распоряжение КГИОП № 10-22 от 21.07.2009,Памятник,POINT (30.31219 59.91660),30.312194,59.916595
1,2,—,Здание манежа (экзерциргауса) лейб-гвардии Изм...,1795-1797,"1-я Красноармейская ул., 13; Измайловский пр.,...",объект культурного наследия регионального знач...,Закон Санкт-Петербурга № 141-47 от 02.07.1997,Памятник,POINT (30.30911 59.91619),30.309113,59.916189
2,3,—,"Дом, где в начале 1895 г. Ленин В.И. встречалс...",—,"1-я Красноармейская ул., 22",объект культурного наследия регионального знач...,Закон Санкт-Петербурга № 141-47 от 02.07.1997,Памятник,POINT (30.31080 59.91592),30.310799,59.915919
3,4,—,Трансформаторная подстанция,1906-1907,"11-я Красноармейская ул., 28",объект культурного наследия регионального знач...,Решение исполкома Ленгорсовета № 963 от 05.12....,Памятник,POINT (30.29881 59.91195),30.298809,59.911953
4,5,—,Манеж графа Г.И. Рибопьера (Здание Санкт-Петер...,1887; 1891,"2-я Красноармейская ул., 12",объект культурного наследия регионального знач...,Распоряжение КГИОП № 10-26 от 15.09.2009,Памятник,POINT (30.31241 59.91520),30.312410,59.915200
...,...,...,...,...,...,...,...,...,...,...,...
1017,9601,—,Здание (многоквартирный дом),1857-1858,"Фонтанки р. наб., 128, лит. А",выявленный объект культурного наследия,Распоряжение КГИОП от 17.06.2019 № 289-р,Памятник,POINT (30.30533 59.91818),30.305331,59.918183
1018,9602,—,Здание (многоквартирный дом),1866,"Фонтанки р. наб., 132, лит. З",выявленный объект культурного наследия,Распоряжение КГИОП от 17.06.2019 № 290-р,Памятник,POINT (30.30340 59.91774),30.303399,59.917736
1019,9608,—,"Участок сохранившегося культурного слоя, вмеща...",—,"квартал, образованный пер. Бойцова, наб. р. Фо...",выявленный объект культурного наследия,Распоряжение КГИОП от 06.08.2019 № 450-р,Памятник,POINT (30.30927 59.92216),30.309268,59.922163
1020,9624,—,Депо комплекса построек Товарной станции Варша...,—,"Обводного кан. наб., 118, корп. 12, лит. Ч",выявленный объект культурного наследия,Распоряжение КГИОП от 22.08.2019 № 507-р,Памятник,POINT (30.30234 59.90612),30.302339,59.906120


In [79]:
# сохраним слой в географической проекции в формате GeoJSON
heritage.to_crs(4326).to_file('heritage.geojson', driver='GeoJSON')